In [ ]:
import numpy as np
import h5py
import pandas as pd
from matplotlib import pyplot as plt
from ipywidgets import interact, IntSlider
import seaborn as sns
from matplotlib.colors import LogNorm
import os
import src

## Set inputs

In [ ]:
# chunk size
chunk_size = 1000

# make a histogram with n bins
n_bins = 500

#tof scaling
tof_scaling = 1e-9

#q scaling: from GeV to eV, from eV to e (3.6eV / e- in Si)
q_scaling = 1e9/3.67

#q threshold
q_threshold = 1000

# window size: extend by this many in both directions
shift_n_times = 0

# tof offset in [ns]
shift_offset = 0

# print infos during run
verbose = False

# width of tornado mask [ns]
mask_xwidth = 25

#shift mask along x axis [ns]
mask_xoffset = -5 

#set to -1 to mirror curves against time; stretch mask along x axis
mask_xscale = -1 

# to handle numerical rounding errors
epsilon = 1e-8

# remote path to hdf5
remote_path = "../../../sshfs/"

## Read hdf5

In [ ]:
hf = h5py.File(remote_path+'tof_q.h5', 'r')

#get group
hf_group = hf.get('Tof_q')
hf_group.keys()

In [ ]:
hf_dset = hf_group.get("kaons+-")
hf_dset

## Bin data

In [ ]:
tof_low = np.inf
tof_high = -np.inf
q_low = np.inf
q_high = -np.inf

# chunks size to read data in chunk_size chunks
n_chunks = int(np.ceil(len(hf_dset)/chunk_size))

#find the overall min/max
# x axis in [ns] (*1e-9) y axis in [GeV] (1e-3)
for chunk_idx in range(n_chunks):
    print("[{}/{}]".format(chunk_idx+1, n_chunks))
    if verbose:
        print("tof min: {}, tof max: {}".format(tof_low, tof_high))
        print("q min: {}, q max: {}".format(q_low, q_high))
    tof_low = np.minimum(hf_dset[chunk_idx*chunk_size:(chunk_idx+1)*chunk_size, 0].min(), tof_low)
    tof_high = np.maximum(hf_dset[chunk_idx*chunk_size:(chunk_idx+1)*chunk_size, 0].max(), tof_high)
    q_low = np.minimum(hf_dset[chunk_idx*chunk_size:(chunk_idx+1)*chunk_size, 1].min(), q_low)
    q_high = np.maximum(hf_dset[chunk_idx*chunk_size:(chunk_idx+1)*chunk_size, 1].max(), q_high)
    
# extend tof range to copy data
tof_high += shift_n_times*shift_offset
tof_low -= shift_n_times*shift_offset

# scale bin limits
tof_low = tof_low*tof_scaling
tof_high = tof_high*tof_scaling
q_low = q_low*q_scaling
q_high = q_high*q_scaling

In [ ]:
# create bin limits based on min/max values, and scale the values
tof_bin_edges = np.linspace(tof_low-epsilon, tof_high+epsilon, (2*shift_n_times+1)*n_bins+1)
q_bin_edges = np.linspace(q_low-epsilon, q_high+epsilon, n_bins+1)

#create empty 2d histograms
tof_histo = np.zeros((n_bins, (2*shift_n_times+1)*n_bins))
tof_below_histo = np.zeros((n_bins, (2*shift_n_times+1)*n_bins))
tof_above_histo = np.zeros((n_bins, (2*shift_n_times+1)*n_bins))

# iterate over the dataset in chunks of n_chunks lines
for chunk_idx in range(n_chunks):
    print("[{}/{}]".format(chunk_idx+1, n_chunks))
    
    # x axis in [ns] (*1e-9) y axis in [GeV] (1e-3)
    tof_chunk = hf_dset[chunk_idx*chunk_size:(chunk_idx+1)*chunk_size, 0]
    q_chunk = hf_dset[chunk_idx*chunk_size:(chunk_idx+1)*chunk_size, 1]
    if verbose:
        print("New data chunk length: {}".format(len(tof_chunk)))
    
    # create data copies (offset value in [ns])
    tof_chunk_extended = np.copy(tof_chunk)
    q_chunk_extended = np.copy(q_chunk)
    for i in range(1, shift_n_times+1):
        tof_chunk_extended = np.concatenate((tof_chunk_extended, tof_chunk+i*shift_offset, tof_chunk-i*shift_offset))
        q_chunk_extended = np.concatenate((q_chunk_extended, q_chunk, q_chunk))    
    tof_chunk = tof_chunk_extended
    q_chunk = q_chunk_extended
    if verbose:
        print("Extended data chunk length: {}".format(len(tof_chunk)))
    
    # scale data
    tof_chunk *= tof_scaling
    q_chunk *= q_scaling
    
    # split data along q threshold
    tof_chunk_below = tof_chunk[q_chunk < q_threshold]
    q_chunk_below = q_chunk[q_chunk < q_threshold]
    tof_chunk_above = tof_chunk[q_chunk >= q_threshold]
    q_chunk_above = q_chunk[q_chunk >= q_threshold]
    
    #fill 2d histo with data chunk
    tof_chunk_histo, xedges, yedges = np.histogram2d(
        tof_chunk, q_chunk, bins=(tof_bin_edges, q_bin_edges));
    tof_chunk_below_histo, xedges_below, yedges_below = np.histogram2d(
        tof_chunk_below, q_chunk_below, bins=(tof_bin_edges, q_bin_edges));
    tof_chunk_above_histo, xedges_above, yedges_above = np.histogram2d(
        tof_chunk_above, q_chunk_above, bins=(tof_bin_edges, q_bin_edges));
    
    # accumulate bin counts over chunks
    tof_histo += tof_chunk_histo.T
    tof_below_histo += tof_chunk_below_histo.T
    tof_above_histo += tof_chunk_above_histo.T
    print("Hits added to histo so far: {}".format(int(np.sum(tof_histo))))

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(np.ma.masked_where(tof_histo==0, tof_histo), interpolation='nearest', origin='lower',
        extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]], aspect="auto", vmin=0, vmax=1000)
plt.grid()
plt.xlabel("ToF [s]", fontsize=16)
plt.ylabel("Q [e-]", fontsize=16)
#plt.ylim(0, .001*q_scaling)

## Load timewalk sim data

In [ ]:
#read timewalk sim
data = pd.read_csv("../tornado_plot/timewalk_Qth1000_RD53B.csv", header=0)

# insert a 0th and a n+1th row to facilitate thresholding at 1000 e-
data = pd.concat([pd.DataFrame(data.iloc[0]).T, data]).reset_index(drop=True)

In [ ]:
#read timewalk sim
data = pd.read_csv("../tornado_plot/timewalk_Qth1000_RD53B.csv", header=0)

# insert a 0th and a n+1th row to facilitate thresholding at 1000 e-
data = pd.concat([pd.DataFrame(data.iloc[0]).T, data]).reset_index(drop=True)
data.at[0, "Timewalk"] = data.at[0, "Timewalk"] + mask_xwidth*tof_scaling
data = pd.concat([data, pd.DataFrame(data.iloc[-1]).T]).reset_index(drop=True)
data.at[len(data)-1, "Qel "] = data.at[len(data)-1, "Qel "]*2

#scale mask e.g. mirror in time
data["Timewalk"] = mask_xscale*data["Timewalk"] + mask_xoffset*tof_scaling

#shift curve to get other boundary
data["timewalk_shifted"] = data["Timewalk"] + mask_xwidth*tof_scaling

data

In [ ]:
#plot
fig = plt.figure(figsize=(10, 10))
for i in range(-shift_n_times, shift_n_times+1):
    l1 = "simulation" if i == 0 else ""
    l2 = "shifted by 25ns" if i == 0 else ""
    plt.plot(data["Timewalk"]+i*shift_offset*tof_scaling, data["Qel "], label=l1, c="b")
    plt.plot(data["timewalk_shifted"]+i*shift_offset*tof_scaling, data["Qel "], label=l2, c="r")
plt.xlabel("Time of flight [s]")
plt.ylabel("Q [e-]")
plt.legend()
plt.grid()

## Overlay sim curve and bib data

In [ ]:
X_mask, Y_mask = np.meshgrid(tof_bin_edges, q_bin_edges)

In [ ]:
plt.figure(figsize=(10, 10))
plt.pcolormesh(X_mask, Y_mask, np.ma.masked_where(tof_histo==0, tof_histo))
#plt.ylim(0, .01*q_scaling)
plt.grid()

# tornado mask curves
plt.plot(data["Timewalk"], data["Qel "], label="simulation")
plt.plot(data["timewalk_shifted"], data["Qel "], label="shifted")
plt.legend()
plt.xlabel("Time of flight [ns]", fontsize=16)
plt.ylabel("Q [e-]", fontsize=16)

### Coordinate geometry way to calculate area below curve
color scatter
offset

In [ ]:
cx_list = []
cy_list = []

cell_mask = np.zeros((n_bins, (2*shift_n_times+1)*n_bins))#*np.nan

#loop goes from 0 to nbins, starts from bottom left corner
for yi in range(n_bins):
    print("[{}/{}]".format(yi+1, n_bins))
    for xi in range((2*shift_n_times+1)*n_bins):
        xmin, xmax = xedges[xi], xedges[xi+1]
        ymin, ymax = yedges[yi], yedges[yi+1]
        cell_area = (xmax - xmin)*(ymax - ymin)            
        cx = (xmax + xmin)/2
        cy = (ymax + ymin)/2

        #get cell area below all curves
        area_below_left_curve = []
        area_below_right_curve = []
        for i in range(-shift_n_times, shift_n_times+1):
            if verbose:
                print("working on mask copy {}".format(i))
            area_below_left_curve.append(src.getAreaBelowCurve(xmin, xmax, ymin, ymax, cx, cy,
                                        data["Timewalk"]+i*shift_offset*tof_scaling, data["Qel "], verbose=verbose))
            area_below_right_curve.append(src.getAreaBelowCurve(xmin, xmax, ymin, ymax, cx, cy,
                                        data["timewalk_shifted"]+i*shift_offset*tof_scaling, data["Qel "], verbose=verbose))
        
        # cell mask value is between 0 and 1
        area_between_curves = np.sum(area_below_left_curve) - np.sum(area_below_right_curve)
        cell_mask_value = area_between_curves/cell_area
        
        #fill up mask array with cell mask value
        if cell_mask_value < epsilon:
            cell_mask_value = 0
        else:
            
            #store cells that are between two curves
            cx_list.append(cx)
            cy_list.append(cy)
            
            #numerics correction
            if cell_mask_value > 1:
                cell_mask_value = 1
            if np.abs(cell_mask_value - 1) < epsilon:
                cell_mask_value = 1   
  
            #save cell masks into numpy array (start from top left corner)
            cell_mask[yi, xi] = cell_mask_value

In [ ]:
%matplotlib

In [ ]:
#data
fig = plt.figure(figsize=(10, 10))
data_plot = plt.pcolormesh(X_mask, Y_mask, np.ma.masked_where(tof_histo==0, tof_histo),
                           vmin=1, vmax=np.max(tof_histo), norm=LogNorm())

# tornado mask bounding curves
#for i in range(-shift_n_times, shift_n_times+1):
#    l1 = "simulation" if i == 0 else ""
#    l2 = "shifted by 25ns" if i == 0 else ""
#    plt.plot(data["Timewalk"]+i*shift_offset*tof_scaling, data["Qel "], label=l1, c="b")
#    plt.plot(data["timewalk_shifted"]+i*shift_offset*tof_scaling, data["Qel "], label=l2, c="r")

#tornado mask
#mask = plt.pcolormesh(X_mask, Y_mask, np.ma.masked_where(cell_mask==-1, cell_mask), alpha=.1, cmap=plt.get_cmap('Greys_r'))

plt.legend()
plt.xlabel("Time of flight [s]", fontsize=16)
plt.ylabel("Q [e-]", fontsize=16)
#plt.ylim(0,80000)
plt.xlim(tof_low, tof_high)
cbar = fig.colorbar(data_plot) 
cbar.set_label('Data count',size=18)
plt.grid()

plt.title("All hits [ {} hits| 100%]".format(int(np.sum(tof_histo))))

## Apply mask to data and produce 3 plots

In [ ]:
set(cell_mask.flatten())

In [ ]:
plt.imshow(cell_mask, origin="lower")

In [ ]:
alpha_hits = src.mcOverlay(cell_mask, tof_above_histo)
beta_hits = tof_above_histo - alpha_hits
gamma_hits = tof_below_histo

In [ ]:
#data
fig = plt.figure(figsize=(10, 10))
data_plot = plt.pcolormesh(X_mask, Y_mask, np.ma.masked_where(alpha_hits==0, alpha_hits),
                           vmin=1, vmax=np.max(tof_histo), norm=LogNorm())

# tornado mask bounding curves
#plt.plot(data["Timewalk"], data["Qel "], label="simulation", c="b")
#plt.plot(data["timewalk_shifted"], data["Qel "], label="shifted by 25ns", c="r")

#tornado mask
#mask = plt.pcolormesh(X_mask, Y_mask, np.ma.masked_where(cell_mask==0, cell_mask), alpha=.6, cmap=plt.get_cmap('viridis'))

plt.legend()
plt.xlabel("Time of flight [s]", fontsize=16)
plt.ylabel("Q [e-]", fontsize=16)
#plt.ylim(0,80000)
plt.xlim(tof_low, tof_high)
cbar = fig.colorbar(data_plot) 
cbar.set_label('Data count',size=18)
plt.grid()

plt.title("Alpha hits [ {} hits| {}%]".format(int(np.sum(alpha_hits)),
                                              round(100*np.sum(alpha_hits)/np.sum(tof_histo), 2)))

#plt.savefig("alpha_hits.png")

In [ ]:
#%matplotlib

#data
fig = plt.figure(figsize=(10, 10))
data_plot = plt.pcolormesh(X_mask, Y_mask, np.ma.masked_where(beta_hits<1e-8, beta_hits),
                           vmin=1, vmax=np.max(tof_histo), norm=LogNorm())

# tornado mask bounding curves
#plt.plot(data["Timewalk"], data["Qel "], label="simulation", c="b")
#plt.plot(data["timewalk_shifted"], data["Qel "], label="shifted by 25ns", c="r")

#tornado mask
#mask = plt.pcolormesh(X_mask, Y_mask, np.ma.masked_where(cell_mask==0, cell_mask), alpha=.6, cmap=plt.get_cmap('viridis'))

plt.legend()
plt.xlabel("Time of flight [s]", fontsize=16)
plt.ylabel("Q [e-]", fontsize=16)
#plt.ylim(0,80000)
plt.xlim(tof_low, tof_high)
cbar = fig.colorbar(data_plot) 
cbar.set_label('Data count',size=18)
plt.grid()

plt.title("Beta hits [ {} hits| {}%]".format(int(np.sum(beta_hits)),
                                             round(100*np.sum(beta_hits)/np.sum(tof_histo), 2)))

#plt.savefig("beta_hits.png")

In [ ]:
#%matplotlib

#data
fig = plt.figure(figsize=(10, 10))
data_plot = plt.pcolormesh(X_mask, Y_mask, np.ma.masked_where(gamma_hits<1e-8, gamma_hits),
                           vmin=1, vmax=np.max(tof_histo), norm=LogNorm())

# tornado mask bounding curves
#plt.plot(data["Timewalk"], data["Qel "], label="simulation", c="b")
#plt.plot(data["timewalk_shifted"], data["Qel "], label="shifted by 25ns", c="r")

#tornado mask
#mask = plt.pcolormesh(X_mask, Y_mask, np.ma.masked_where(cell_mask==0, cell_mask), alpha=.6, cmap=plt.get_cmap('viridis'))

plt.legend()
plt.xlabel("Time of flight [s]", fontsize=16)
plt.ylabel("Q [e-]", fontsize=16)
#plt.ylim(0,80000)
plt.xlim(tof_low, tof_high)
cbar = fig.colorbar(data_plot) 
cbar.set_label('Data count',size=18)
plt.grid()

plt.title("Gamma hits [ {} hits| {}%]".format(int(np.sum(gamma_hits)),
                                              round(100*np.sum(gamma_hits)/np.sum(tof_histo), 2)))

#plt.savefig("gamma_hits.png")

In [ ]:
hf.close()